In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from os import getcwd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

In [2]:
train_dir = 'C:/Users/Asus/COVID19 datasets - Limited (3 classes)/Training'
validation_dir = 'C:/Users/Asus/COVID19 datasets - Limited (3 classes)/Testing'

train_pneu_dir = train_dir+'/PNEUMONIA'+'/train/'
train_covid_dir = train_dir+'/COVID19'+'/train/'
train_normal_dir = train_dir+'/NORMAL'+'/train/'

validation_pneu_dir = validation_dir+'/PNEUMONIA'+'/test/'
validation_covid_dir = validation_dir+'/COVID19'+'/test/'
validation_normal_dir = validation_dir+'/NORMAL'+'/test/'

In [3]:
ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    verbose=0,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0
)

In [4]:
class myCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.96):
            print("\nReached limit accuracy so cancelling training!")
            self.model.stop_training = True

In [5]:
batch = 12

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size = batch,
                                                   class_mode = 'categorical',
                                                   target_size = (224, 224),
                                                   shuffle = True)     

validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                        batch_size = batch,
                                                        class_mode = 'categorical',
                                                        target_size = (224, 224),
                                                        shuffle = True)

Found 750 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [6]:
local_weights_file = f"{getcwd()}/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

pre_trained_model = Xception(include_top = False, 
                             weights = None,
                             input_shape = (224, 224, 3),
                             pooling = 'avg')

pre_trained_model.load_weights(local_weights_file)

pre_trained_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [7]:
for layer in pre_trained_model.layers:
    layer.trainable = False

In [8]:
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 2048)


In [9]:
x = AveragePooling2D((4, 4)) (last_output)
x = Flatten()(x)
x = Dense(64, activation = 'relu')(x)
x = Dropout(0.5)(x)                  
x = Dense(3, activation = 'softmax')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = Adam(learning_rate = 0.001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [10]:
callbacks = myCallback()
reduce_lr = ReduceLROnPlateau()
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    verbose = 1,
                    epochs = 40,
                    batch_size = batch,
                    callbacks = [callbacks, reduce_lr])

Epoch 1/40
63/63 [==============================] - 477s 7s/step - loss: 0.5065 - accuracy: 0.7707 - val_loss: 0.4012 - val_accuracy: 0.8267 - lr: 0.0010
Epoch 2/40
63/63 [==============================] - 630s 10s/step - loss: 0.3274 - accuracy: 0.8760 - val_loss: 0.2902 - val_accuracy: 0.8867 - lr: 0.0010
Epoch 3/40
63/63 [==============================] - 462s 7s/step - loss: 0.2416 - accuracy: 0.9067 - val_loss: 0.2302 - val_accuracy: 0.9267 - lr: 0.0010
Epoch 4/40
63/63 [==============================] - 177s 3s/step - loss: 0.2052 - accuracy: 0.9333 - val_loss: 0.2777 - val_accuracy: 0.8933 - lr: 0.0010
Epoch 5/40
63/63 [==============================] - 154s 2s/step - loss: 0.1848 - accuracy: 0.9387 - val_loss: 0.1965 - val_accuracy: 0.9400 - lr: 0.0010
Epoch 6/40
63/63 [==============================] - 179s 3s/step - loss: 0.1430 - accuracy: 0.9493 - val_loss: 0.1823 - val_accuracy: 0.9467 - lr: 0.0010
Epoch 7/40
63/63 [==============================] - 182s 3s/step - loss: 0.

In [11]:
results = model.evaluate(validation_generator, batch_size=batch)
print("test loss, test acc:", results)

13/13 [==============================] - 15s 1s/step - loss: 0.2025 - accuracy: 0.9400
test loss, test acc: [0.20249319076538086, 0.9399999976158142]


In [12]:
10 epochs - > 0.8133

SyntaxError: invalid syntax (2585528530.py, line 1)

In [ ]:
len(pre_trained_model.layers)